In [ ]:
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
import numpy as np
import time
from bioservices.kegg import KEGG
from biomart import BiomartServer
from io import StringIO

# Define clusters for analysis
cluster_pairs = [
    ('Silver nitrate', 'Sorbitol', 'Freeze-Thaw', 'Aluminum chloride', 'Potassium chloride', 'Hydrogen peroxide', 'Heat Stress'),
    ('Cobalt', 'Nickel', 'Vanillin', 'Propolis'),
    ('Ethanol', 'Boron', 'Sodium acetate', 'Phenyl ethanol', 'Chromium(III) chloride', 'Ammonium iron(III) sulfate', 'Manganese(II) chloride'),
    ('Methanol', 'Magnesium chloride', 'Acetate'),
    ('Copper', 'Sodium chloride'),
    ('Rapamycin', 'Caffeine', 'Coniferyl aldehyde')
]

# Dictionary to store unique and significant genes for each cluster
unique_genes = {}
significant_genes = {}

# Helper function to split a list into chunks of size n
def split_list(lst, n):
    """Split the list into chunks of size n."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Function to fetch gene information from Biomart
def get_gene_info(gene_ids):
    """Fetch gene details like GO terms and descriptions from Biomart."""
    # Connect to the Biomart server
    server = BiomartServer("http://www.ensembl.org/biomart")
    server.verbose = True
    
    # Select the yeast dataset
    dataset = server.datasets['scerevisiae_gene_ensembl']
    
    # Split gene list into manageable chunks
    all_chunks = list(split_list(gene_ids, 50))
    dfs = []
    
    for chunk in all_chunks:
        try:
            # Query Biomart for gene information
            response = dataset.search({
                'filters': {
                    'ensembl_gene_id': chunk
                },
                'attributes': [
                    'ensembl_gene_id',
                    'external_gene_name',
                    'description',
                    'go_id',  # Gene Ontology ID
                    'name_1006',  # GO term name
                ]
            })
            
            # Convert response to DataFrame
            data = response.text
            if data.strip():  # If data exists
                df = pd.read_csv(StringIO(data), sep='\t', header=None, names=['ensembl_gene_id', 'external_gene_name', 'description', 'go_id', 'go_term'])
                dfs.append(df)
            
            # Pause to avoid overloading the server
            time.sleep(1)
        except Exception as e:
            print(f"Error fetching data for chunk: {chunk} - {e}")
    
    # Concatenate all chunks into a final DataFrame
    if dfs:
        final_df = pd.concat(dfs, ignore_index=True)
    else:
        final_df = pd.DataFrame(columns=['ensembl_gene_id', 'external_gene_name', 'description', 'go_id', 'go_term'])
    
    return final_df

# Function to convert gene IDs to gene names, descriptions, and GO terms
def convert_gene_ids(gene_ids):
    """Convert Ensembl gene IDs to gene names and descriptions using Biomart."""
    gene_info_df = get_gene_info(gene_ids)
    
    # Group by ensembl_gene_id and aggregate GO terms
    gene_info_df_grouped = gene_info_df.groupby('ensembl_gene_id').agg({
        'external_gene_name': 'first',
        'description': 'first',
        'go_id': lambda x: '; '.join(x.dropna().unique()),
        'go_term': lambda x: '; '.join(x.dropna().unique())
    }).reset_index()
    
    # Convert to dictionary format
    gene_info_dict = gene_info_df_grouped.set_index('ensembl_gene_id').to_dict(orient='index')
    return gene_info_dict

# Function for KEGG pathway analysis
def kegg_pathway_analysis(gene_list):
    """Retrieve KEGG pathways for a list of genes."""
    kegg = KEGG()
    kegg.organism = "sce"  # KEGG organism code for Saccharomyces cerevisiae
    
    pathway_dict = {}
    for gene in gene_list:
        try:
            # Retrieve KEGG pathways associated with the gene
            pathways = kegg.get_pathway_by_gene(gene, organism="sce")
            if pathways:
                pathway_dict[gene] = pathways
        except Exception as e:
            print(f"Error processing gene {gene}: {e}")
    
    return pathway_dict

# Loop through each cluster for analysis
for cluster in cluster_pairs:
    unique_genes[cluster] = {'upregulated': [], 'downregulated': []}
    significant_genes[cluster] = {'upregulated': [], 'downregulated': []}
    
    cluster_samples = list(cluster)  # Samples for the current cluster
    other_samples = [col for col in df.columns if col not in cluster_samples]  # Samples not in the cluster

    # Loop through each gene in the dataframe
    for gene in df.index:
        cluster_values = df.loc[gene, cluster_samples]  # Values for cluster samples
        other_values = df.loc[gene, other_samples]  # Values for other samples

        # Check for unique expression pattern in the cluster
        if (cluster_values != 0).all() and (other_values == 0).all():
            if (cluster_values > 0).all():
                unique_genes[cluster]['upregulated'].append(gene)
            elif (cluster_values < 0).all():
                unique_genes[cluster]['downregulated'].append(gene)
        else:
            # Perform normality test
            if len(cluster_values) >= 3 and len(other_values) >= 3:
                cluster_normal = shapiro(cluster_values)[1] > 0.05
                other_normal = shapiro(other_values)[1] > 0.05

                # Apply t-test or Mann-Whitney U test based on normality
                if cluster_normal and other_normal:
                    stat, p_value = ttest_ind(cluster_values, other_values)
                else:
                    stat, p_value = mannwhitneyu(cluster_values, other_values)

                # If p-value is significant, classify as upregulated or downregulated
                if p_value < 0.05:
                    if cluster_values.mean() > other_values.mean():
                        significant_genes[cluster]['upregulated'].append(gene)
                    else:
                        significant_genes[cluster]['downregulated'].append(gene)

# Convert gene IDs to gene names and descriptions
all_genes = set()
for cluster in cluster_pairs:
    all_genes.update(unique_genes[cluster]['upregulated'])
    all_genes.update(unique_genes[cluster]['downregulated'])
    all_genes.update(significant_genes[cluster]['upregulated'])
    all_genes.update(significant_genes[cluster]['downregulated'])

gene_info_dict = convert_gene_ids(list(all_genes))

# Save results to CSV
for cluster in cluster_pairs:
    # Create a DataFrame for output
    output_df = pd.DataFrame(columns=['Category', 'Gene', 'Gene Name', 'Description', 'GO Terms', 'KEGG Pathways'])
    
    def add_to_output(category, genes):
        """Helper function to add genes to the output DataFrame."""
        for gene in genes:
            if gene in gene_info_dict:
                go_id = gene_info_dict[gene].get('go_id', '')
                go_term = gene_info_dict[gene].get('go_term', '')
                go_info = f"GO: {go_id} - {go_term}" if go_id or go_term else ''
                
                try:
                    kegg_info = "; ".join(kegg_pathway_analysis([gene_info_dict[gene]['external_gene_name']])[gene_info_dict[gene]['external_gene_name']])
                except KeyError:
                    kegg_info = ''

                output_df.loc[len(output_df)] = [
                    category,
                    gene,
                    gene_info_dict[gene]['external_gene_name'],
                    gene_info_dict[gene]['description'],
                    go_info,
                    kegg_info
                ]
            else:
                print(f"Gene {gene} not found in gene_info_dict.")
    
    # Add genes from each category to the output DataFrame
    add_to_output('Unique Upregulated Genes', unique_genes[cluster]['upregulated'])
    add_to_output('Unique Downregulated Genes', unique_genes[cluster]['downregulated'])
    add_to_output('Significant Upregulated Genes', significant_genes[cluster]['upregulated'])
    add_to_output('Significant Downregulated Genes', significant_genes[cluster]['downregulated'])
    
    # Save the output to CSV
    output_df.to_csv(f'{cluster}_significant_genes.csv', index=False)

print("Results have been saved to CSV files.")

# CSV file for enrichment analysis
csv_file_path = 'path/filename.csv'
df = pd.read_csv(csv_file_path)

# Add an enrichment column
df['Enrichment'] = ""

# Function to fetch KEGG pathway information for a gene
def get_pathway_info(gene):
    """Retrieve KEGG pathway descriptions for a given gene."""
    try:
        pathways = kegg.get_pathway_by_gene(gene, organism="sce")
        return "; ".join(pathways.values()) if pathways else ""
    except Exception as e:
        return f"Error retrieving KEGG data for {gene}: {e}"

# Apply KEGG pathway analysis
df['Enrichment'] = df['Gene'].apply(lambda gene: get_pathway_info(gene))

# Save the updated DataFrame with KEGG pathways
df.to_csv(f'{csv_file_path}_with_kegg.csv', index=False)
